In [ ]:
# python version 3.10.13
# Author : Venugopal

import folium
m = folium.Map()
m = folium.Map(location=(60.63879, 15.44528))
m

In [3]:
import pandas as pd
lake_chem = pd.read_csv('C:/Users/vijay/Documents/projects/dataScienceProject/lakechemdataclean.csv')
lake_chem_lat_long = lake_chem.drop_duplicates(subset=['Latitude', 'Longitude'],keep='first')

In [ ]:
# add marker one by one on the map to locate the lake collection points

for i in range(0,len(lake_chem_lat_long)):
   folium.Marker(
      location=[lake_chem_lat_long.iloc[i]['Latitude'], lake_chem_lat_long.iloc[i]['Longitude']],
      popup=lake_chem_lat_long.iloc[i]['Survey station'],
   ).add_to(m)

m


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

#Extracting the longitude and latitude from the files to create geometry of the shapefile for that particular year and season
df_y05s01 = pd.read_csv(r"C:\Users\vijay\Documents\projects\dataScienceProject\data\year_season\y05_s01.csv") 
df_y05s01_shp = gpd.GeoDataFrame(df_y05s01, crs='EPSG:4326', geometry=gpd.points_from_xy(df_y05s01['Longitude'], df_y05s01['Latitude']))

# saving the created shapefile in the destination directory
filepath = r"C:/Users/vijay/Documents/projects/dataScienceProject/data/year_season/shapefiles"
df_y05s01_shp.to_file(f"{filepath}/y05s01.shp", driver='ESRI Shapefile')

In [ ]:
''' 
#Alternative way to achieve the same results
geometry = [Point(xy) for xy in zip(df_landscape['Longitude'], df_landscape['Latitude'])]
gdf = GeoDataFrame(df_landscape, geometry=geometry)
gdf.set_crs(epsg=4326, inplace=True)
gdf.to_crs(epsg=3395)

gdf

'''

In [ ]:
'''
# Data processing for clipping according to the year and season
import pandas as pd


df = pd.read_csv(r"C:\Users\vijay\Documents\projects\dataScienceProject\lakechemdataclean.csv")
df.head()

y22_s04 = df.loc[(df['Year'] == 2022) & (df['Season'] == 4)]
y22_s04.to_csv(r"C:\Users\vijay\Documents\projects\dataScienceProject\data\year_season\y22_s04.csv")

'''

In [ ]:
''' 
# Cleaning the data for NaN or Inf

import pandas as pd
df = pd.read_csv(r"E:\year_season\y20_s04.csv")
df.columns[df.isna().any()].tolist()

# Dropping the columns which contain NaN values
drop_columns = ['cpyll', 'oxy', 'water_temp']
df = df.drop(drop_columns, axis=1)


# Saving the cleaned file to destination location
filepath=r"E:/"
df.to_csv(f"{filepath}/y20s04.csv")

'''

In [ ]:
# Normalizing the lake chemistry data

import pandas as pd

df = pd.read_csv(r"E:\year_season\y05_s01.csv")
#df.info()

df = df.drop(['md_mvm_id',
              'lat','long',
              'year', 'month', 'day',
              'season_type', 'season',
              'max_sample_depth'],
               axis=1)

df_ = df.fillna(df.mean())
df_normalized = (df_ - df_.mean())/df_.std()  

df_normalized.isnull().any(axis=1)

In [ ]:
# Processing and data manipulation for PCA

import pandas as pd

df = pd.read_csv(r"E:\year_season_copy\y05s01_shapefile\y05_s01.csv")
#df.head()
df_copy = df.copy()

df_for_normalization = df_copy.drop(['md_mvm_id','lat','long','year', 'month','day','season_type','season'], axis=1)

df = df_for_normalization.fillna(df_for_normalization.mean())
df_ = df.drop(['toc', 'abs420'], axis=1)
df_



In [ ]:
df_normalized = (df_ - df_.mean()) / df_.std()
df_normalized

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plot

pca = PCA(n_components=10)
pca.fit(df_normalized)

In [ ]:
pca.components_
pca_df = pd.DataFrame(pca.components_)
pca_df

In [ ]:
pca = PCA(n_components=df_.shape[1])
pca.fit(df_normalized)


In [ ]:
# Reformat and view results

loadings = pandas.DataFrame(pca.components_.T,
columns=['PC%s' % _ for _ in range(len(df_normalized.columns))],
index=df_.columns)
print(loadings)

In [ ]:
# Plotting the variance vs components

plot.plot(pca.explained_variance_ratio_)
plot.ylabel('Explained Variance')
plot.xlabel('Components')
plot.show()

In [ ]:
from pca import pca

model = pca(n_components=0.95)
results = model.fit_transform(df_normalized)
fig, ax = model.plot()

In [ ]:
# Bi-plot
model.biplot(cmap=None, legend=1, n_feat=12)

In [ ]:
# print the principle component loadings
print(model.results['topfeat'])

In [ ]:
# Random forest Implementation part

import pandas as pd

path_to_data = r"E:\year_season_copy\y05s01_shapefile\y05_s01.csv"
dataset = pd.read_csv(path_to_data)
dataset.head()

In [ ]:
# data Cleaning and normalizing

drop_columns  =['md_mvm_id','lat','long','abs420','year','month', 'day', 'season_type', 'season']
dataset = dataset.drop(drop_columns, axis=1)


data_median = dataset.median()
dataset = dataset.fillna(data_median)

In [ ]:
# correlation in the data

dataset.corr()

In [ ]:
#plotting the heatmap

import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


sns.set(style="white")
corr = dataset.corr()
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True
f, ax = plt.subplots(figsize=(20, 19))
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5)


In [ ]:
# checking for NaN values before modelling

dataset.isna().values.any()

In [ ]:
# Assigning the Dependent and Independent variables

labels = ['toc']
y = dataset[labels].values
X = dataset.drop(labels, axis=1).values

y.shape, X.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [ ]:
from sklearn.preprocessing import RobustScaler

scl_X = RobustScaler().fit(X_train)
scl_y = RobustScaler().fit(y_train)
input_x_train = scl_X.transform(X_train)
input_y_train = scl_y.transform(y_train)
input_x_test = scl_X.transform(X_test)
input_y_test = scl_y.transform(y_test)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor

In [ ]:
linreg = MultiOutputRegressor(LinearRegression())
linreg.fit(input_x_train, input_y_train)

In [ ]:
#Training score for linear regression
linreg.score(input_x_train, input_y_train)

In [ ]:
#Testing score for linear regression
linreg.score(input_x_test, input_y_test)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor

In [ ]:
#Random forest modelling
RF = MultiOutputRegressor(RandomForestRegressor(n_estimators=200, max_features="sqrt", max_depth=10, min_samples_leaf=7))
RF.fit(input_x_train, input_y_train)

In [ ]:
#training
RF.score(input_x_train, input_y_train)

In [ ]:
#testing
RF.score(input_x_test, input_y_test)

In [ ]:
# Decision Tree Regressor implementation part

In [ ]:
from sklearn.tree import DecisionTreeRegressor   #algorithm
from sklearn.ensemble import RandomForestClassifier   #algorithm
from sklearn.ensemble import GradientBoostingClassifier   #algorithm

from sklearn.model_selection import train_test_split   #split dataset to train and test

from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV

from sklearn.tree import DecisionTreeRegressor

import matplotlib.pyplot as plt   #for regurlar plots
import pandas as pd
import numpy as np

#settings
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
#split data
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=42)

In [ ]:
#build model
tree = DecisionTreeRegressor(random_state=0)
tree.fit(X_train, y_train)
print("Accuracy on training set: {:.3f}".format(tree.score(X_train, y_train)))
print("Accuracy on test set: {:.3f}".format(tree.score(X_test, y_test)))

In [ ]:
param_grid = {
    'max_depth': [80, 90, 100, None],
    'max_features': ["sqrt", "log2", None],
    'min_samples_leaf': [1, 3, 5],
    'min_samples_split': [2, 3, 4],
    'criterion': ["squared_error", "friedman_mse", "absolute_error", "poisson"],
    'splitter': ["best", "random"]
}

rf = DecisionTreeRegressor()

grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 5, verbose =2, scoring='r2',  n_jobs = -1)
grid_search.fit(X_train, y_train)

In [ ]:
# searching for the best parameters to fit
grid_search.best_params_

In [ ]:
#fine tuning model

tree = DecisionTreeRegressor(random_state=42, criterion='absolute_error', max_depth=80, max_features=None,min_samples_leaf=5, min_samples_split=3,splitter="random")
tree.fit(X_train, y_train)
print("Accuracy on training set: {:.3f}".format(tree.score(X_train, y_train)))
print("Accuracy on test set: {:.3f}".format(tree.score(X_test, y_test)))